# 3 - Times Series

## Generate Series

In [ ]:
import pandas as pd

df = pd.read_csv('data/_________________.csv', sep=';', header=0)

df["Quarter"] = pd.to_datetime(df["Quarter"])
dtindex = pd.DatetimeIndex(data=df["Quarter"], freq='infer') # create Pandas DateTimeIndex

df.set_index(dtindex, inplace=True) # Set as Index
df.drop("Quarter", axis=1, inplace=True) # Drop original column
df.head()

# optionally plot
import matplotlib.pyplot as plt
df.plot(y="_________________")
plt.show()



### Calculate Log Returns and Plot


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

df = df.astype('float')
df["rel"] = np.log(df) - np.log(df.shift(1))

df.plot(y="rel")

## Calculate Trend

In [ ]:
df["Trend"] = df["________"].rolling(window=4, center=True).mean()

In [ ]:
# and plot
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1)
df.plot(y="Trend", ax=ax)
ax.set_xlabel("time")
ax.set_ylabel("Beer Prodiction [Ml]")
ax.grid()

#### Plot nach Quartal  

In [ ]:
import seaborn as sns
# Plot using Seaborn
fig, ax = plt.subplots(figsize=(8, 5))
df['quarter'] = df.index.quarter
sns.boxplot(data=df, x='quarter', y="________")
plt.show()


## Seasonal Decomposition

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomp = seasonal_decompose(df["________"],
                            model = "additive", # "additive" or "multiplicative"
                            period=4) ## --> 4 für Quartal
# Plot
fig = decomp.plot()
fig.set_size_inches(12, 6)
plt.show()

## Seasonal Decomposition with STL

In [ ]:
from statsmodels.tsa.seasonal import STL
# Decomposition on log-model using STL
decomp = STL(df["________"], seasonal=5)
decomp = decomp.fit()
# Plot
fig = decomp.plot()
fig.set_size_inches(12, 6)
plt.show()

## BoxCox Transformation

In [ ]:
# version 1) 
df_sq = (np.sqrt(df) - 1) * 2

In [ ]:
# Boxcox Definition:
def boxcox(x, lambd):
    if (lambd == 0):
        return np.log(x)
    else:
        return (x ** lambd - 1) / lambd
    

# Apply transform
df["l_025"] =   boxcox(df["kilowatt"], 0.25)
df["l_05"] =    boxcox(df["kilowatt"], 0.5)
df["l_075"] =   boxcox(df["kilowatt"], 0.75)
# Plots
fig, ax = plt.subplots(figsize=(12, 8), nrows=2, ncols=2)
df.plot(ax=ax[:], subplots=True)
plt.show()


## ACF / PACF Plots

In [ ]:
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf

data = force_train
lags = 50

if lags > len(data):
    lags = len(data) - 1

# Plot
fig = plt.figure(figsize=(14, 5))
ax1 = fig.add_subplot(1, 2, 1)
plot_acf(data, lags=50, ax=ax1)
ax2 = fig.add_subplot(1, 2, 2)
plot_pacf(data, lags=50, ax=ax2)
ax2.plot([9, 9], [-0.76, 1], ':r')

plt.show()

# FIT AR Model


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
order = 4
data = df

model = ARIMA(data, order=(order, 0, 0))
model = model.fit(method="yule_walker")

print(model.summary())

In [ ]:
# Plot
from statsmodels.graphics.api import qqplot

fig = plt.figure(figsize=(12, 5))
ax1 = fig.add_subplot(1, 2, 1)
model.resid.hist(edgecolor="black", bins=50, ax=ax1)
plt.xlabel("Residual")
plt.ylabel("Frequency")
ax2 = fig.add_subplot(1, 2, 2)
qqplot(model.resid, line="q", ax=ax2)
plt.show()


# Prediction durch Autoregression

In [ ]:
data = df

pred = model.get_prediction(start="2017", end="2117") # for the next 100 years
pred = pred.prediction_results
pred_cov = pred._forecasts_error_cov
pred = pred._forecasts[0]
pred_upper = pred + 1.96 * np.sqrt(pred_cov[0][0])
pred_lower = pred - 1.96 * np.sqrt(pred_cov[0][0])

# Plot
x = data.index.year
x_pred = np.arange(2017, 2118)
fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1)
ax.plot(x, data, '-k', label='train data')
ax.plot(x_pred, pred, 'r', label='Prediction')
ax.plot(x_pred, pred_upper, ':b', label='95% conf')
ax.plot(x_pred, pred_lower, ':b')
ax.plot([x[0], x_pred[-1]], [0, 0], ':k')
plt.xlabel("Time")
plt.ylabel("Values")
plt.legend()
plt.show()

# Prediction durch Moving Average    

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(data, order=(0, 0, q))  # z. B. q = 2
model = model.fit()